In [1]:
# Instalamos pyspark y py4j
!pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=cff019413c6b7f127d6b5b34dd459b3c52fb4d81e4daea3a3df0f88b6e3e2197
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

Spark = SparkSession.builder.appName("lion").getOrCreate()
sc = Spark.sparkContext

**1-. Cree un RDD llamado lenguajes que contenga los siguientes lenguajes de programación: Python, #R, C, Scala, Rugby y SQL.**
- A) Cree un nuevo RDD que solo contenga aquellos lenguajes de programación que comiencen con la letra R.


In [3]:
rdd_lenguajes = sc.parallelize(["Python", "R", "C", "Scala", "Rugby", "SQL"])
rdd_lenguajes.collect()

['Python', 'R', 'C', 'Scala', 'Rugby', 'SQL']

**A.-**

In [4]:
rdd_lenguajes_con_r = rdd_lenguajes.filter(lambda x: x.startswith("R"))
rdd_lenguajes_con_r.collect()

['R', 'Rugby']


**2-. Cree un RDD llamado pares que contenga los números pares existentes en el intervalo [20;30].**
- B) Obtenga una lista compuesta por los números pares en el intervalo [20;30] y sus respectivas raíces cuadradas. Un ejemplo del resultado deseado para el intervalo [50;60] sería la lista [50, 7.0710678118654755, 52, 7.21110255092 54,7.3484692283495345, 56, 7.483314773547883, 58, 7.615773105863909, 60, 7.745966692414834

- C) Eleve el número de particiones del RDD sqrt a 20.

- D) Si tuviera que disminuir el número de particiones luego de haberlo
establecido en 20, ¿qué función utilizaría para hacer más eficiente su código?



In [5]:
numeros = list(range(20, 30))
rdd_numeros = sc.parallelize(numeros)

rdd_pares = rdd_numeros.filter(lambda x: x % 2 == 0)

rdd_pares.collect()

[20, 22, 24, 26, 28]

**B.-**

In [8]:
rdd_sqrt = rdd_pares.flatMap(lambda x: [x, x ** 0.5])
rdd_sqrt.collect()

[20,
 4.47213595499958,
 22,
 4.69041575982343,
 24,
 4.898979485566356,
 26,
 5.0990195135927845,
 28,
 5.291502622129181]

**C.-**

In [10]:
rdd_eleve = rdd_sqrt.repartition(20)
rdd_eleve.getNumPartitions()

20

**D.-**

In [12]:
#   RESPUESTA: Para disminuir el numero de particiones usaria la funcion coalesce, ya que este no afecta los datos.

rdd_dismi = rdd_eleve.coalesce(10)
rdd_dismi.getNumPartitions()

10

**3-. Cree un RDD del tipo clave valor a partir de la lectura del TXT de transacciones.**
- Tenga en cuenta que deberá procesar el RDD leído para obtener el resultado solicitado.
- Supongamos que el RDD resultante de tipo clave valor refleja las transacciones realizadas por número de cuentas.
- Obtenga el monto total por cada cuenta. (Suma)

Tip: Cree su propia función para procesar el RDD leído.

In [14]:
leer = '/content/transacciones.txt'
rdd_lineas = sc.textFile(leer)

rdd_mostrar = rdd_lineas.map(lambda line: ((line.split()[0], line.split()[1])))

rdd_mostrar.collect()


[('(1001,', '52.3)'),
 ('(1005,', '20.8)'),
 ('(1001,', '10.1)'),
 ('(1004,', '52.7)'),
 ('(1005,', '20.7)'),
 ('(1002,', '85.3)'),
 ('(1004,', '20.9)')]

In [17]:
# Función para extraer números con variable llamada "extraer_numeros" de una cadena y convertirlos en floats.

#========================= AGREGUE un def y variable "extraer_numeros" para poder tener los numeros mas limpios =========================
#==========================================       y borrar los simbolos del resultado       =============================================

def extraer_numeros(line):
    valores = line.replace('(', '').replace(')', '').replace('-', '').split(',')
    try:
        return (valores[0], float(valores[1]))
    except Error:
        return None

#======================================================================================================================================

rdd_numeros = rdd_lineas.map(extraer_numeros).filter(lambda x: x is not None)

rdd_resultado = rdd_numeros.reduceByKey(lambda x, y: x + y).collect()

for clave, suma in rdd_resultado:
    print(f"Numero de Cuenta: {clave} - Suma Total: {suma}")

Numero de Cuenta: 1002 - Suma Total: 85.3
Numero de Cuenta: 1001 - Suma Total: 62.4
Numero de Cuenta: 1005 - Suma Total: 41.5
Numero de Cuenta: 1004 - Suma Total: 73.6
